In [29]:
from selenium import webdriver
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import random
import time
# Need to build out the scraper to scrape all stats and join tables on player name while eliminating duplicate columns


player_data = []

url = "https://fbref.com/en/comps/22/stats/Major-League-Soccer-Stats"

driver = webdriver.Chrome()

driver.get(url)

driver.implicitly_wait(10)

html = driver.page_source

soup = BeautifulSoup(html, 'html.parser')
league = "MLS"
# Get stats table
div_stats_standard = soup.find('div', {'id': 'div_stats_standard'}) 
if div_stats_standard != None:

    # Table Headers
    headers = []
    header_row = div_stats_standard.find('thead').find('tr').find_next('tr')
    for header in header_row.find_all('th'):
        headers.append(header.text.strip())
    headers.pop(0)
    # Table Data
    table_data = []
    table_body = div_stats_standard.find('tbody')
    count = 0
    playerCount = 0
    for row in table_body.find_all('tr'):
        row_data = []
        for cell in row.find_all('td'):
            row_data.append(cell.text.strip())
        #table_data.append(row_data)
        columns = row.find_all('td')  # Replace 'td' with the actual tag or class name of the table cell
        if len(columns) > 1:  # Make sure the row has enough columns
            base_url = "https://fbref.com/"
            url =  columns[0].find('a')['href']  # Replace '1' with the column index containing the hrefs
            href = f"{base_url}{url}"
            print("Link:", href)
            wait_time = random.uniform(1, 3)  # Generates a random wait time between 1 and 3 seconds
            time.sleep(wait_time)
            # Step 5: Fetch the linked webpage and parse it
            linked_response = requests.get(href)
            linked_html_content = linked_response.text
            linked_soup = BeautifulSoup(linked_html_content, 'html.parser')
            
            text = linked_soup.get_text()  # Get the text content of the table

            # Find the index of "Position:"
            position_index = text.find("Position:")
            born_index = text.find("Born:")
            if position_index != -1:
                # Extract the text following "Position:"
                position_text = text[position_index + len("Position:") :born_index].strip()
                
                #print("Position:", position_text)
                if position_text.find("Footed:") != -1:
                    position_text = position_text[:position_text.find("Footed:")].strip()
                row_data.append(position_text)
                if playerCount == 0:
                    headers.append("Advanced Position")
            # Step 6: Find the tables in the linked webpage
            linked_tables = linked_soup.find_all('table')  # Replace 'table' with the actual tag or class name of the table element
            for linked_table in linked_tables:
                table_id = linked_table.get('id')  # Get the ID attribute of the table
                #print("TestTable ID:", table_id)
                validIds = ["stats_standard_dom_lg", "stats_shooting_dom_lg", "stats_passing_dom_lg", "stats_passing_types_dom_lg", "stats_gca_dom_lg", "stats_defense_dom_lg", "stats_possession_dom_lg","stats_playing_time_dom_lg", "stats_misc_dom_lg"]
                if table_id in validIds:
                    #if table_id:
                        #print("Table ID:", table_id)
                    # Find the row with "Season" column value equals "2023"
                    linked_rows = linked_table.find_all('tr')  # Replace 'tr' with the actual tag or class name of the table row
                    for linked_row in linked_rows:
                        linked_columns = linked_row.find_all('th')  # Replace 'td' with the actual tag or class name of the table cell
                        season_column = linked_columns[0]  # Replace '0' with the column index containing the "Season" column
                        if season_column.get_text().strip() == "2023":
                            # Append the values from the linked row to the original row
                            for linked_col in linked_columns:
                                #row_data = []
                                for cell in linked_row.find_all('td'):
                                    if len(row_data) <= len(headers):
                                        row_data.append(cell.text.strip())
                    # Table Headers
                    #headers2 = []
                    if playerCount == 0:
                        header_row = linked_table.find('thead').find('tr').find_next('tr')
                        hCount = 0
                        for header in header_row.find_all('th'):
                            if hCount != 0:
                                headers.append(header.text.strip())
                            hCount += 1
                    #headers2.pop(0)
                    #headers.append(headers2)
                    #print(len(headers))
        table_data.append(row_data)
                            #columns.append(linked_col)
        playerCount += 1
        count +=1
        print(count)
        #if count >= 5:
        #    break
                
    #print(headers)
    #print(table_data)
    df = pd.DataFrame(table_data, columns=headers)

    player_data.append(df)
    naming = "mls23testfull.csv"

    df.to_csv(naming)
    print(url)
    print("Done with", league)

Link: https://fbref.com//en/players/1f462f95/Luis-Abram
1
Link: https://fbref.com//en/players/4a67f05b/Lalas-Abubakar
2
Link: https://fbref.com//en/players/4ab21ca4/Daniel-Aceves
3
Link: https://fbref.com//en/players/c5f400f4/Bryan-Acosta
4
Link: https://fbref.com//en/players/ece10cfe/Kellyn-Acosta
5
Link: https://fbref.com//en/players/363ba6d7/Luciano-Acosta
6
Link: https://fbref.com//en/players/86429529/Samuel-Adeniran
7
Link: https://fbref.com//en/players/7d8ede09/Harrison-Afful
8
Link: https://fbref.com//en/players/cea02b5b/Stephen-Afrifa
9
Link: https://fbref.com//en/players/d2275b1b/William-Agada
10
Link: https://fbref.com//en/players/ecc5063f/Daniel-Aguirre
11
Link: https://fbref.com//en/players/d987142b/Patrick-Agyemang
12
Link: https://fbref.com//en/players/540a5be1/Ali-Ahmed
13
Link: https://fbref.com//en/players/f2a68b97/Carlos-Akapo
14
Link: https://fbref.com//en/players/cfed08d5/Matai-Akinmboni
15
Link: https://fbref.com//en/players/214d2406/Ayo-Akinola
16
Link: https://fb

136
Link: https://fbref.com//en/players/cc120b4f/Julio-Cascante
137
Link: https://fbref.com//en/players/f86fe8c1/Cristian-Casseres-Jr
138
Link: https://fbref.com//en/players/e620dcbb/Robert-Castellanos
139
Link: https://fbref.com//en/players/42ba8628/Roman-Celentano
140
Link: https://fbref.com//en/players/bd060315/Edwin-Cerrillo
141
Link: https://fbref.com//en/players/04cc9edd/Maikel-Chang
142
Link: https://fbref.com//en/players/ca366055/Maxime-Chanot
143
Link: https://fbref.com//en/players/6260cc9d/Diego-Chara
144
Link: https://fbref.com//en/players/9df9baab/Yimmi-Chara
145
Link: https://fbref.com//en/players/e85fb9b8/Giorgio-Chiellini
146
Link: https://fbref.com//en/players/017459bb/Mathieu-Choiniere
147
Link: https://fbref.com//en/players/d88ebbe9/Leo-Chu
148
Link: https://fbref.com//en/players/8c8c07a7/Jose-Cifuentes
149
Link: https://fbref.com//en/players/ed6320b7/Steve-Clark
150
Link: https://fbref.com//en/players/4c46e404/Noah-Cobb
151
Link: https://fbref.com//en/players/00eb5be

271
Link: https://fbref.com//en/players/c644e140/Justin-Haak
272
Link: https://fbref.com//en/players/77d28e45/Luke-Haakenson
273
Link: https://fbref.com//en/players/582952f2/Teenage-Hadebe
274
Link: https://fbref.com//en/players/39632554/Nick-Hagglund
275
Link: https://fbref.com//en/players/aee414b1/Maren-Haile-Selassie
276
Link: https://fbref.com//en/players/6ad5f50e/Michael-Halliday
277
Link: https://fbref.com//en/players/fc7c8d3d/Ahmed-Hamdi
278
Link: https://fbref.com//en/players/b149b4ad/Yosuke-Hanya
279
Link: https://fbref.com//en/players/4a78c769/Cameron-Harper
280
Link: https://fbref.com//en/players/b3edbeff/Nathan-Harriel
281
Link: https://fbref.com//en/players/07b0aeef/Calvin-Harris
282
Link: https://fbref.com//en/players/86bac87b/Thomas-Hasal
283
Link: https://fbref.com//en/players/d552c4a1/Heber
284
Link: https://fbref.com//en/players/5e2a4d0e/Matt-Hedges
285
286
Link: https://fbref.com//en/players/99a6142d/Chris-Hegardt
287
Link: https://fbref.com//en/players/5cbf5ccd/Fabi

406
Link: https://fbref.com//en/players/565fde4a/Anthony-Markanich
407
Link: https://fbref.com//en/players/e75f9dba/George-Marks
408
Link: https://fbref.com//en/players/01802cf6/Mikael-Marques
409
Link: https://fbref.com//en/players/b9459f85/Jahkeele-Marshall-Rutty
410
Link: https://fbref.com//en/players/8e049cbd/Jose-Martinez
411
Link: https://fbref.com//en/players/e73f0aa1/Josef-Martinez
412
Link: https://fbref.com//en/players/b7cc83e3/Celio-Martins
413
Link: https://fbref.com//en/players/56b7c09b/Luis-Martins
414
Link: https://fbref.com//en/players/4d6ec370/Thiago-Martins
415
416
Link: https://fbref.com//en/players/3bd3bcc4/Alexandru-Matan
417
Link: https://fbref.com//en/players/ca1c6f3d/Jimmy-Maurer
418
Link: https://fbref.com//en/players/fda36cfb/Chris-Mavinga
419
Link: https://fbref.com//en/players/67c3efa4/Max
420
Link: https://fbref.com//en/players/52fdf512/Andreas-Maxso
421
Link: https://fbref.com//en/players/f6315619/Olivier-Mbaizo
422
Link: https://fbref.com//en/players/df4f

541
Link: https://fbref.com//en/players/4c53a714/Kacper-Przybylko
542
Link: https://fbref.com//en/players/f090a37a/Riqui-Puig
543
Link: https://fbref.com//en/players/b3085df7/Alan-Pulido
544
Link: https://fbref.com//en/players/6b306c92/John-Pulskamp
545
546
Link: https://fbref.com//en/players/5b33b84c/Facundo-Quignon
547
Link: https://fbref.com//en/players/a295f0b3/Nelson-Quinones
548
Link: https://fbref.com//en/players/ad411288/Philip-Quinton
549
Link: https://fbref.com//en/players/122e850d/Romell-Quioto
550
Link: https://fbref.com//en/players/b5dd3469/Nemanja-Radoja
551
Link: https://fbref.com//en/players/09507ffa/Aleksandar-Radovanovic
552
Link: https://fbref.com//en/players/a3d67f70/Jackson-Ragen
553
Link: https://fbref.com//en/players/b4caa0c5/Brooklyn-Raines
554
Link: https://fbref.com//en/players/316d5521/Christian-Ramirez
555
Link: https://fbref.com//en/players/f006a9b3/Ryan-Raposo
556
Link: https://fbref.com//en/players/989ecf23/Justin-Rasmussen
557
Link: https://fbref.com//en

677
Link: https://fbref.com//en/players/7e1046fa/Thorleifur-Ulfarsson
678
Link: https://fbref.com//en/players/e7056f05/Victor-Ulloa
679
Link: https://fbref.com//en/players/6c8fb474/Maximiliano-Urruti
680
Link: https://fbref.com//en/players/93b56ba5/Leo-Vaisanen
681
Link: https://fbref.com//en/players/3fbe573e/Jhojan-Valencia
682
Link: https://fbref.com//en/players/f0d23e7b/Zarek-Valentin
683
Link: https://fbref.com//en/players/2ce88662/Gustavo-Vallecilla
684
Link: https://fbref.com//en/players/d682f554/Dante-Vanzeir
685
Link: https://fbref.com//en/players/758de7b4/Kerwin-Vargas
686
Link: https://fbref.com//en/players/e8d67346/Marlon-Vargas
687
Link: https://fbref.com//en/players/9cc1fde4/Obed-Vargas
688
Link: https://fbref.com//en/players/04e195ee/Indiana-Vassilev
689
Link: https://fbref.com//en/players/c570cf22/Brandon-Vazquez
690
Link: https://fbref.com//en/players/6c564d36/Victor-Vazquez
691
Link: https://fbref.com//en/players/e0cd04e0/Carlos-Vela
692
Link: https://fbref.com//en/pla

ValueError: 274 columns passed, passed data had 511 columns

In [31]:
print(len(headers))

274


In [17]:
print(headers)

['Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls', 'Ast', 'G+A', 'G-PK', 'G+A-PK', 'xG', 'xAG', 'xG+xAG', 'npxG', 'npxG+xAG', 'Matches', 'Age', 'Squad', 'Country', 'Comp', 'LgRank', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls', 'Ast', 'G+A', 'G-PK', 'G+A-PK', 'xG', 'xAG', 'xG+xAG', 'npxG', 'npxG+xAG', 'Matches', 'Age', 'Squad', 'Country', 'Comp', 'LgRank', 'MP', 'Min', 'Mn/MP', 'Min%', '90s', 'Starts', 'Mn/Start', 'Compl', 'Subs', 'Mn/Sub', 'unSub', 'PPM', 'onG', 'onGA', '+/-', '+/-90', 'On-Off', 'onxG', 'onxGA', 'xG+/-', 'xG+/-90', 'On-Off', 'Matches', 'Age', 'Squad', 'Country', 'Comp', 'LgRank', '90s', 'CrdY', 'CrdR', '2CrdY', 'Fls', 'Fld', 'Off', 'Crs', 'Int', 'TklW', 'PKwon', 'PKcon', 'OG', 'Recov', 'Won', 'Lost', 'Won%', '

In [40]:
copyOf = table_data
cdf = pd.DataFrame(copyOf)
cdf = cdf.iloc[:, 0:274]
cdf.columns = headers
cdf
naming = "mls23full.csv"

cdf.to_csv(naming)

In [24]:
for i in range(145):
    print(headers[i])
    print(df[i][0])

Player
Luis Abram
Nation
pe PER
Pos
DF
Squad
Atlanta Utd
Age
27-094
Born
1996
MP
7
Starts
1
Min
122
90s
1.4
Gls
0
Ast
0
G+A
0
G-PK
0
PK
0
PKatt
0
CrdY
0
CrdR
0
xG
0.0
npxG
0.0
xAG
0.0
npxG+xAG
0.0
PrgC
2
PrgP
6
PrgR
0
Gls
0.00
Ast
0.00
G+A
0.00
G-PK
0.00
G+A-PK
0.00
xG
0.00
xAG
0.00
xG+xAG
0.00
npxG
0.00
npxG+xAG
0.00
Matches
Matches
Age
DF (CB, left) ▪
Squad
Luis Abram
Country
pe PER
Comp
DF
LgRank
Atlanta Utd
MP
27-094
Starts
1996
Min
7
90s
1
Gls
122
Ast
1.4
G+A
0
G-PK
0
PK
0
PKatt
0
CrdY
0
CrdR
0
xG
0
npxG
0
xAG
0.0
npxG+xAG
0.0
PrgC
0.0
PrgP
0.0
PrgR
2
Gls
6
Ast
0
G+A
0.00
G-PK
0.00
G+A-PK
0.00
xG
0.00
xAG
0.00
xG+xAG
0.00
npxG
0.00
npxG+xAG
0.00
Matches
0.00
Age
0.00
Squad
Matches
Country
Luis Abram
Comp
pe PER
LgRank
DF
MP
Atlanta Utd
Min
27-094
Mn/MP
1996
Min%
7
90s
1
Starts
122
Mn/Start
1.4
Compl
0
Subs
0
Mn/Sub
0
unSub
0
PPM
0
onG
0
onGA
0
+/-
0
+/-90
0.0
On-Off
0.0
onxG
0.0
onxGA
0.0
xG+/-
2
xG+/-90
6
On-Off
0
Matches
0.00
Age
0.00
Squad
0.00
Country
0.00
Comp
0.00
LgRank
0.0

IndexError: list index out of range